In [21]:
import struct
import uuid
import binascii
import enum
import datetime as dt
import numpy as np
import queue

In [22]:
packet = bytes.fromhex('')

In [23]:
class binaryPacket:
    def __init__(self, payload:bytes, packetClass:int, packetID:int, sourceUUID:uuid.UUID, destUUID:uuid.UUID):
        self._payload = payload
        self._class = packetClass
        self._id = packetID
        self._source = sourceUUID
        self._dest = destUUID
    
    def to_bytes(self):
        payloadLen = len(self._payload)
        header = struct.pack("<BB", 0xE4, 0xEB) + \
                             self._source.bytes + \
                             self._dest.bytes + \
                             struct.pack("<BBH", self._class, self._id, payloadLen)
        pktCksum = binascii.crc_hqx(header, 0xFFFF).to_bytes(2, "big")
        msg = header + pktCksum + self._payload
        cksum = binascii.crc_hqx(msg, 0xFFFF).to_bytes(2, "big")
        return msg + cksum
    
    def getClassIDCode(self):
        return (self._class << 8) | self._id
    
    def __str__(self):
        string = self.to_bytes().hex().upper()
        length = 4
        return '%s' % ' '.join(string[i:i + length] for i in range(0, len(string), length))
    
    def __repr__(self):
        return str(self)
    
    def __eq__(self, packet):
        return self.to_bytes() == packet.to_bytes()
    
    @classmethod
    def from_bytes(cls, packet:bytes):
        srcUUID, destUUID, pcls, pid, payload = cls.parseHeader(packet)
        return binaryPacket(payload, pcls, pid, uuid.UUID(bytes=srcUUID), uuid.UUID(bytes=destUUID))
    
    @classmethod
    def parseHeader(cls, packet:bytes):
        if binascii.crc_hqx(packet, 0xFFFF) != 0:
            raise RuntimeError("Checksum verification failed")
            
        if binascii.crc_hqx(packet[0:0x0028], 0xFFFF) != 0:
            raise RuntimeError("Header checksum verification failed")
        
        if len(packet) < 0x2A:
            raise RuntimeError("Packet too short!")
            
            
        s1, s2, pcls, pid, _ = struct.unpack("<BB16x16xBBH", packet[0:0x26])
        srcUUID = bytes(packet[0x0002:0x0012])
        destUUID = bytes(packet[0x0012:0x0022])
        if s1 != 0xE4 or s2 != 0xEB:
            raise RuntimeError("Not a packet!")
        payload = packet[0x28:-2]
        return srcUUID, destUUID, pcls, pid, payload
    
    @classmethod
    def matches(cls, packetClass:int, packetID:int):
        return True

In [24]:
class E4E_Data_IMU(binaryPacket):
    __class = 0x04
    __id = 0x00
    __VERSION = 0x01

    def __init__(self, src:uuid.UUID, dest:uuid.UUID, accX, accY, accZ, gyroX, gyroY, gyroZ, magX, magY, magZ, timestamp:dt.datetime = None):
        self.acc = [accX, accY, accZ]
        self.gyro = [gyroX, gyroY, gyroZ]
        self.mag = [magX, magY, magZ]
        if timestamp is None:
            timestamp = dt.datetime.now()
        self.timestamp = timestamp
        
        
        payload = struct.pack("<BBQ3f3f3f",
                                   self.__VERSION,
                                   0x00,
                                   int(timestamp.timestamp() * 1e3),
                                   *self.acc,
                                   *self.gyro,
                                   *self.mag)
        super(E4E_Data_IMU, self).__init__(payload, self.__class, self.__id, src, dest)
        
    @classmethod
    def matches(cls, packetClass:int, packetID:int):
        return packetClass == self.__class and packetID == self.__id
    
    @classmethod
    def from_bytes(cls, packet:bytes):
        srcUUID, destUUID, pcls, pid, payload = cls.parseHeader(packet)
        ver, _, timestamp_ms, accX, accY, accZ, gyroX, gyroY, gyroZ, magX, magY, magZ = struct.unpack("<BBQ3f3f3f", payload)
        if ver != cls.__VERSION:
            return RuntimeError("Unknown packet version!")
        
        timestamp = dt.datetime.fromtimestamp(timestamp_ms / 1e3)
        return E4E_Data_IMU(src=uuid.UUID(bytes=srcUUID),
                           dest=uuid.UUID(bytes=destUUID),
                           accX = accX,
                           accY = accY,
                           accZ = accZ,
                           gyroX = gyroX,
                           gyroY = gyroY,
                           gyroZ = gyroZ,
                           magX = magX,
                           magY = magY,
                           magZ = magZ,
                           timestamp=timestamp)

In [25]:
class E4E_Data_Audio_raw8(binaryPacket):
    __class = 0x04
    __id = 0x01
    __VERSION = 0x01
    
    def __init__(self, audioData:np.ndarray, src:uuid.UUID, dest:uuid.UUID, timestamp:dt.datetime=None):
        assert(len(audioData.shape) == 2)
        nChannels = audioData.shape[0]
        nSamples = audioData.shape[1]
        if timestamp is None:
            timestamp = dt.datetime.now()
        
        self.audioData = audioData
        payload = struct.pack("<BBHQ",
                             self.__VERSION,
                             nChannels,
                              nSamples,
                              int(timestamp.timestamp() * 1e3))
        for channel in range(nChannels):
            for sampleIdx in range(nSamples):
                payload += struct.pack("<B", int(audioData[channel, sampleIdx]))
        super(E4E_Data_Audio_raw8, self).__init__(payload, self.__class, self.__id, src, dest)
    
    @classmethod
    def matches(cls, packetClass:int, packetID:int):
        return packetClass == self.__class and packetID == self.__id
    
    @classmethod
    def from_bytes(cls, packet:bytes):
        srcUUID, destUUID, pcls, pid, payload = cls.parseHeader(packet)
        ver, nChannels, nSamples, timestamp_ms = struct.unpack("<BBHQ", payload[0:0x0C])
        audioBytes = payload[0x0C:]
        assert(len(audioBytes) == nChannels * nSamples)
        idx = 0
        audioData = np.zeros((nChannels, nSamples))
        for channel in range(nChannels):
            for sampleIdx in range(nSamples):
                audioData[channel, sampleIdx] = audioBytes[idx]
                idx += 1
        timestamp = dt.datetime.fromtimestamp(timestamp_ms / 1e3)
        return E4E_Data_Audio_raw8(audioData, uuid.UUID(bytes=srcUUID), uuid.UUID(bytes=destUUID), timestamp)

In [26]:
class E4E_Data_Audio_raw16(binaryPacket):
    __class = 0x04
    __id = 0x02
    __VERSION = 0x01
    
    def __init__(self, audioData:np.ndarray, src:uuid.UUID, dest:uuid.UUID, timestamp:dt.datetime=None):
        assert(len(audioData.shape) == 2)
        nChannels = audioData.shape[0]
        nSamples = audioData.shape[1]
        if timestamp is None:
            timestamp = dt.datetime.now()
        
        self.audioData = audioData
        payload = struct.pack("<BBHQ",
                             self.__VERSION,
                             nChannels,
                              nSamples,
                              int(timestamp.timestamp() * 1e3))
        for channel in range(nChannels):
            for sampleIdx in range(nSamples):
                payload += struct.pack("<H", int(audioData[channel, sampleIdx]))
        super(E4E_Data_Audio_raw16, self).__init__(payload, self.__class, self.__id, src, dest)
    
    @classmethod
    def matches(cls, packetClass:int, packetID:int):
        return packetClass == self.__class and packetID == self.__id
    
    @classmethod
    def from_bytes(cls, packet:bytes):
        srcUUID, destUUID, pcls, pid, payload = cls.parseHeader(packet)
        ver, nChannels, nSamples, timestamp_ms = struct.unpack("<BBHQ", payload[0:0x0C])
        audioBytes = payload[0x0C:]
        assert(len(audioBytes) == nChannels * nSamples * 2)
        idx = 0
        audioData = np.zeros((nChannels, nSamples))
        for channel in range(nChannels):
            for sampleIdx in range(nSamples):
                audioData[channel, sampleIdx], = struct.unpack("<H", audioBytes[idx * 2 : idx * 2 + 2])
                idx += 1
        timestamp = dt.datetime.fromtimestamp(timestamp_ms / 1e3)
        return E4E_Data_Audio_raw16(audioData, uuid.UUID(bytes=srcUUID), uuid.UUID(bytes=destUUID), timestamp)

In [27]:
class E4E_Data_Raw_File_Header(binaryPacket):
    __class = 0x04
    __id = 0xFC
    __VERSION = 0x01
    
    def __init__(self, fileID:int, filename:str, MIMEType:str, fileSize:int, fileTime:dt.datetime, src:uuid.UUID, dest:uuid.UUID):
        self.fileID = fileID
        self.filename = filename
        self.mimeType = MIMEType
        self.fileSize = fileSize
        self.fileTime = fileTime
        payload = struct.pack("<BBHHQQ",
                             self.__VERSION,
                             fileID,
                             len(filename),
                             len(MIMEType),
                             fileSize,
                             int(fileTime.timestamp() * 1e3))
        payload += filename.encode('ascii')
        payload += MIMEType.encode('ascii')
        super(E4E_Data_Raw_File_Header, self).__init__(payload, self.__class, self.__id, src, dest)
        
    @classmethod
    def matches(cls, packetClass:int, packetID:int):
        return packetClass == self.__class and packetID == self.__id

    @classmethod
    def from_bytes(cls, packet:bytes):
        srcUUID, destUUID, pcls, pid, payload = cls.parseHeader(packet)
        ver, fileID, filenameLen, mimeTypeLen, fileSize, fileTimestamp = struct.unpack("<BBHHQQ", payload[0:0x16])
        filename = payload[0x16:0x16 + filenameLen].decode()
        mimeType = payload[0x16 + filenameLen:].decode()
        timestamp = dt.datetime.fromtimestamp(fileTimestamp / 1e3)
        return E4E_Data_Raw_File_Header(fileID, filename, mimeType, fileSize, timestamp, uuid.UUID(bytes=srcUUID), uuid.UUID(bytes=destUUID))

In [28]:
class E4E_Data_Raw_File_CTS(binaryPacket):
    __class = 0x04
    __id = 0xFD
    __VERSION = 0x01
    
    def __init__(self, fileID:int, ack:bool, src:uuid.UUID, dest:uuid.UUID):
        self.fileID = fileID
        self.ack = ack
        payload = struct.pack("<BBB", self.__VERSION, fileID, int(ack))
        super(E4E_Data_Raw_File_CTS, self).__init__(payload, self.__class, self.__id, src, dest)
        
    @classmethod
    def matches(cls, packetClass:int, packetID:int):
        return packetClass == self.__class and packetID == self.__id

    @classmethod
    def from_bytes(cls, packet:bytes):
        srcUUID, destUUID, pcls, pid, payload = cls.parseHeader(packet)
        fileID = payload[1]
        if payload[2] == 1:
            ack = True
        else:
            ack = False
        return E4E_Data_Raw_File_CTS(fileID, ack, uuid.UUID(bytes=srcUUID), uuid.UUID(bytes=destUUID))

In [29]:
class E4E_Data_Raw_File_ACK(binaryPacket):
    __class = 0x04
    __id = 0xFE
    __VERSION = 0x01
    
    def __init__(self, fileID:int, seq:int, ack:bool, src:uuid.UUID, dest:uuid.UUID):
        self.fileID = fileID
        self.seq = seq
        self.ack = ack
        payload = struct.pack("<BBHB", self.__VERSION, fileID, seq, int(ack))
        super(E4E_Data_Raw_File_ACK, self).__init__(payload, self.__class, self.__id, src, dest)
        
    @classmethod
    def matches(cls, packetClass:int, packetID:int):
        return packetClass == self.__class and packetID == self.__id

    @classmethod
    def from_bytes(cls, packet:bytes):
        srcUUID, destUUID, pcls, pid, payload = cls.parseHeader(packet)
        ver, fileID, seq, ackInt = struct.unpack("<BBHB", payload)
        if ackInt == 1:
            ack = True
        else:
            ack = False
        return E4E_Data_Raw_File_ACK(fileID, seq, ack, uuid.UUID(bytes=srcUUID), uuid.UUID(bytes=destUUID))

In [30]:
class E4E_Data_Raw_File(binaryPacket):
    __class = 0x04
    __id = 0xFF
    __VERSION = 0x01
    
    def __init__(self, fileID:int, seq:int, blob:bytes, src:uuid.UUID, dest:uuid.UUID):
        self.fileID = fileID
        self.seq = seq
        self.blob = blob
        payload = struct.pack("<BBHQ", self.__VERSION, fileID, seq, len(blob))
        payload += blob
        super(E4E_Data_Raw_File, self).__init__(payload, self.__class, self.__id, src, dest)
        
    @classmethod
    def matches(cls, packetClass:int, packetID:int):
        return packetClass == self.__class and packetID == self.__id

    @classmethod
    def from_bytes(cls, packet:bytes):
        srcUUID, destUUID, pcls, pid, payload = cls.parseHeader(packet)
        ver, fileID, seq, blobLen = struct.unpack("<BBHQ", payload[0:0x0C])
        blob = payload[0x0C:0x0C + blobLen]
        return E4E_Data_Raw_File(fileID, seq, blob, uuid.UUID(bytes=srcUUID), uuid.UUID(bytes=destUUID))

In [31]:
class binaryPacketParser:
    class State(enum.Enum):
        FIND_SYNC1 = 0
        FIND_SYNC2 = 1
        HEADER = 2
        HEADER_CKSUM = 3
        HEADER_VALIDATE = 4
        PAYLOAD = 5
        CKSUM = 6
        VALIDATE = 7
        RECYCLE = 8
    packetMap = {
        0x0400: E4E_Data_IMU,
        0x0401: E4E_Data_Audio_raw8,
        0x0402: E4E_Data_Audio_raw16,
        0x04FC: E4E_Data_Raw_File_Header,
        0x04FD: E4E_Data_Raw_File_CTS,
        0x04FE: E4E_Data_Raw_File_ACK,
        0x04FF: E4E_Data_Raw_File
    }
    
    HEADER_LEN = 0x0026
    
    def __init__(self):
        self.__state = self.State.FIND_SYNC1
        self.__payloadLen = 0
        self.__buffer = None
        self.__data = queue.Queue()
        
    def parseByte(self, data:int):
        self.__data.put(data)
        while not self.__data.empty():
            retval = self._parseByte()
        return retval
            
    def _parseByte(self):
        data = self.__data.get_nowait()
        print("%s: 0x%02x" % (self.__state, data))

        if self.__state is self.State.FIND_SYNC1:
            if data == 0xE4:
                self.__state = self.State.FIND_SYNC2
                self.__buffer = bytearray()
                self.__buffer.append(data)
            return None
        elif self.__state is self.State.FIND_SYNC2:
            if data == 0xEB:
                self.__state = self.State.HEADER
                self.__buffer.append(data)
            else:
                self.__state = self.State.FIND_SYNC1
            return None
        elif self.__state is self.State.HEADER:
            self.__buffer.append(data)
            if len(self.__buffer) == self.HEADER_LEN:
                self.__state = self.State.HEADER_CKSUM
                self.__payloadLen, = struct.unpack('<H', self.__buffer[self.HEADER_LEN - 2:self.HEADER_LEN])
            return None
        elif self.__state is self.State.HEADER_CKSUM:
            self.__buffer.append(data)
            self.__state = self.State.HEADER_VALIDATE
        elif self.__state is self.State.HEADER_VALIDATE:
            self.__buffer.append(data)
            if binascii.crc_hqx(self.__buffer, 0xFFFF) != 0:
#                 print(self.__buffer.hex())
#                 print(binascii.crc_hqx(self.__buffer, 0xFFFF).to_bytes(2, "big").hex())
#                 print("Invalid header")
                # Recycle everything
                self.__state = self.State.FIND_SYNC1
                self.__recycleBuffer = self.__buffer[2:]
                while not self.__data.empty():
                    self.__recycleBuffer.append(self.__data.get_nowait())
                for byte in self.__recycleBuffer:
                    self.__data.put(byte)
            else:
                self.__state = self.State.PAYLOAD
            return None
        elif self.__state is self.State.PAYLOAD:
            self.__buffer.append(data)
            if len(self.__buffer) == self.__payloadLen + self.HEADER_LEN + 2:
                self.__state = self.State.CKSUM
            return None
        elif self.__state is self.State.CKSUM:
            self.__buffer.append(data)
            self.__state = self.State.VALIDATE
            return None
        elif self.__state is self.State.VALIDATE:
            self.__buffer.append(data)
            if binascii.crc_hqx(self.__buffer, 0xFFFF) != 0:
#                 print(self.__buffer.hex())
#                 print(binascii.crc_hqx(self.__buffer, 0xFFFF).to_bytes(2, "big").hex())
                raise RuntimeError("Checksum verification failed")
            packetID, = struct.unpack('>H', self.__buffer[0x0022:0x0024])
            self.__state = self.State.FIND_SYNC1
            if packetID not in self.packetMap:
                return binaryPacket.from_bytes(self.__buffer)
            else:
                return self.packetMap[packetID].from_bytes(self.__buffer)
    
    def parseBytes(self, data:bytes):
        packets = []
        for byte in data:
            self.__data.put(byte)
        while not self.__data.empty():
            retval = self._parseByte()
            if retval is not None:
                packets.append(retval)
        return packets

In [32]:
imuPacket = E4E_Data_IMU(src=uuid.uuid4(),
                        dest=uuid.uuid4(),
                        accX=1.0,
                        accY=2.0,
                        accZ=3.0,
                        gyroX=4.0,
                        gyroY=5.0,
                        gyroZ=6.0,
                        magX=7.0,
                        magY=8.0,
                        magZ=9.0,
                        timestamp=dt.datetime(2020, 12, 24, 0, 13, 13))
print(imuPacket)
print(hex(len(imuPacket.to_bytes())))
parser = binaryPacketParser()
packets = parser.parseBytes(imuPacket.to_bytes())
print(packets)
print(type(packets[0]))

E4EB D6D9 A3F8 554B 4963 A5AD 219C E80D ADD7 276D B630 7B5D 4018 9391 02CC 72BC 3AAF 0400 2E00 7EB3 0100 A81D CF93 7601 0000 0000 803F 0000 0040 0000 4040 0000 8040 0000 A040 0000 C040 0000 E040 0000 0041 0000 1041 BA7B
0x58
State.FIND_SYNC1: 0xe4
State.FIND_SYNC2: 0xeb
State.HEADER: 0xd6
State.HEADER: 0xd9
State.HEADER: 0xa3
State.HEADER: 0xf8
State.HEADER: 0x55
State.HEADER: 0x4b
State.HEADER: 0x49
State.HEADER: 0x63
State.HEADER: 0xa5
State.HEADER: 0xad
State.HEADER: 0x21
State.HEADER: 0x9c
State.HEADER: 0xe8
State.HEADER: 0x0d
State.HEADER: 0xad
State.HEADER: 0xd7
State.HEADER: 0x27
State.HEADER: 0x6d
State.HEADER: 0xb6
State.HEADER: 0x30
State.HEADER: 0x7b
State.HEADER: 0x5d
State.HEADER: 0x40
State.HEADER: 0x18
State.HEADER: 0x93
State.HEADER: 0x91
State.HEADER: 0x02
State.HEADER: 0xcc
State.HEADER: 0x72
State.HEADER: 0xbc
State.HEADER: 0x3a
State.HEADER: 0xaf
State.HEADER: 0x04
State.HEADER: 0x00
State.HEADER: 0x2e
State.HEADER: 0x00
State.HEADER_CKSUM: 0x7e
State.HEADER_VALIDAT

In [33]:
rng = np.random.default_rng()
audioData = rng.integers(255, size=(2, 256))
audioPacket = E4E_Data_Audio_raw8(audioData = audioData,
                                 src=uuid.uuid4(),
                                 dest=uuid.uuid4(),
                                 timestamp=dt.datetime(2020, 12, 24, 20, 12, 8))
print(audioPacket)
print(hex(len(audioPacket.to_bytes())))
packets = parser.parseBytes(audioPacket.to_bytes())
print(packets)
print(type(packets[0]))

E4EB 3C5A 7317 FABC 497A A369 51DE 2BEC 4F9C A0F5 30D8 A12F 46C5 A795 8E75 CC00 8BC6 0401 0C02 70F5 0102 0001 C0C1 1898 7601 0000 9251 4FCF 6D8C 0E38 42D0 3048 A5B3 7FD3 482A 612E 0A7D 4F91 3821 1919 0B4E F786 E774 67A8 4821 BFF7 C47A BFB4 D871 7399 8E92 A38C 163A A104 FE09 9C92 995E 9A63 288B 85FB D4FD F43E 76A0 E091 1677 C15F 3C74 D69D 4602 32B9 6EF4 7645 64C9 25F4 F566 5FE9 43CD FAC9 C19D AB94 4D20 912D D415 00C2 F1C3 3962 97C2 E099 3720 56FC A49F 273B E7A8 B043 44E0 579C C33E 2CF9 8241 DB3F 9F8E 56C9 6B5F 8FBC C42F C57E 77AA F15A A19E 0B6C 65A6 F6A4 D152 EABC 46EA 29BD BCE0 CFDC 1B41 A570 864B 6CD7 6E94 5B71 E852 858E 2214 C7A8 12A8 0B0C 0C5B 3019 0C4D D969 8AB5 456F 8ACE DD00 D0C8 C5BC 24FD 01EB 73CC A841 32F8 175B E9FE 1E7E 8687 BC2B 4033 8F53 1680 C7AC B39E 5C7E B885 A25A 5E69 522D C1A4 6171 F68E FB88 CD99 F301 7552 B144 4328 12C8 9584 6B62 1658 EE88 0E48 E3C0 3791 3989 F6EB 1F5C 5994 7948 28DE 72A3 AE23 9A0F D97A E801 5B47 2E04 1055 FC7E 5174 BC9E EACA A299 7DBC 67C8 CD31 F135 

In [34]:
rng = np.random.default_rng()
audioData = rng.integers(32767, size=(2, 256))
audioPacket = E4E_Data_Audio_raw16(audioData = audioData,
                                 src=uuid.uuid4(),
                                 dest=uuid.uuid4(),
                                 timestamp=dt.datetime(2020, 12, 24, 20, 12, 8))
print(audioPacket)
print(hex(len(audioPacket.to_bytes())))
packets = parser.parseBytes(audioPacket.to_bytes())
print(packets)
print(type(packets[0]))

E4EB E9E6 82A1 E13E 4484 8134 08ED D246 AED4 726A EFC6 2B0E 4614 90EA EC05 59F1 EB1F 0402 0C04 0C47 0102 0001 C0C1 1898 7601 0000 4A2D 6251 7961 280F 4753 8135 6447 CF6F 893F 456C F43D 0557 4612 F401 B46B CF1B FE14 4229 5630 F33C 2375 D27A 344C 1F4B BB76 B77C 7C23 B711 976C 2F31 B170 2E33 F06D 274A F702 E352 3C4B E932 3B24 811D A135 C811 1C23 F237 5240 FC1A DF6B 142E 715A CF74 E03D F47F 3D03 EE7D 7E36 325C 847D BD4B ED34 4402 3636 DC3F 3F78 9D73 B63C 3125 1450 E208 D527 0413 1C02 9A2D 9246 3520 0843 C917 9C2C 613F EE5D 2452 FA3F DC2B F57D 262B 4902 FD7D D051 7C29 2C0B CB5E 8D2A 3D5D E30A 9B56 C421 7366 A947 BB5A AC09 B373 2564 1718 E57B F36E 1825 9969 FB20 896A 911F D055 D425 C954 A979 3B5B 5010 5268 925C E96E 906B C375 C24B 9871 1B5C 3D3A 315B 0B29 6001 2E7D F31B 347F 5D14 2132 7F23 C55E 826D 8457 8132 F823 6C64 1852 8012 395C D851 1C38 1F62 E310 7A09 EC0D 2D29 293C 977C FD58 631F E149 A03E D929 792D 192C 3762 BC3F 7514 983B 114F C916 8E70 A439 1A56 C047 8E47 2B59 BF72 077E 111E 8511 

In [35]:
import os
rng = np.random.default_rng()

fileID = 0
fileName = "Binary Data Protocol Specification.ipynb"
mimeType = "application/x-ipynb+json"
src = uuid.uuid4()
dest = uuid.uuid4()

statResult = os.stat(fileName)
fileSize = statResult.st_size
fileTime = dt.datetime.fromtimestamp(statResult.st_ctime)

packet = E4E_Data_Raw_File_Header(fileID, fileName, mimeType, fileSize, fileTime, src, dest)
print(packet)
print(hex(len(packet.to_bytes())))
packets = parser.parseBytes(packet.to_bytes())
print(packets)
print(type(packets[0]))

E4EB FD01 4ADB 95AC 4DD7 9D92 089B 11AB 99A6 FC0C 9D7F 737A 4C65 BE29 1338 526F D0B5 04FC 5600 9AC0 0100 2800 1800 DB43 0000 0000 0000 8A92 4928 7701 0000 4269 6E61 7279 2044 6174 6120 5072 6F74 6F63 6F6C 2053 7065 6369 6669 6361 7469 6F6E 2E69 7079 6E62 6170 706C 6963 6174 696F 6E2F 782D 6970 796E 622B 6A73 6F6E 9248
0x80
State.FIND_SYNC1: 0xe4
State.FIND_SYNC2: 0xeb
State.HEADER: 0xfd
State.HEADER: 0x01
State.HEADER: 0x4a
State.HEADER: 0xdb
State.HEADER: 0x95
State.HEADER: 0xac
State.HEADER: 0x4d
State.HEADER: 0xd7
State.HEADER: 0x9d
State.HEADER: 0x92
State.HEADER: 0x08
State.HEADER: 0x9b
State.HEADER: 0x11
State.HEADER: 0xab
State.HEADER: 0x99
State.HEADER: 0xa6
State.HEADER: 0xfc
State.HEADER: 0x0c
State.HEADER: 0x9d
State.HEADER: 0x7f
State.HEADER: 0x73
State.HEADER: 0x7a
State.HEADER: 0x4c
State.HEADER: 0x65
State.HEADER: 0xbe
State.HEADER: 0x29
State.HEADER: 0x13
State.HEADER: 0x38
State.HEADER: 0x52
State.HEADER: 0x6f
State.HEADER: 0xd0
State.HEADER: 0xb5
State.HEADER: 0x04
St

In [36]:
rng = np.random.default_rng()
src = uuid.uuid4()
dest = uuid.uuid4()

fileID = 0
ack = True

packet = E4E_Data_Raw_File_CTS(fileID, ack, src, dest)
print(packet)
print(hex(len(packet.to_bytes())))
packets = parser.parseBytes(packet.to_bytes())
print(packets)
print(type(packets[0]))

E4EB B210 073D BB3A 499F 94ED AC42 1921 4D2E 2EE8 AFC0 D1F6 42A3 8C4F DC27 FD0B 87D1 04FD 0300 0B33 0100 0127 11
0x2d
State.FIND_SYNC1: 0xe4
State.FIND_SYNC2: 0xeb
State.HEADER: 0xb2
State.HEADER: 0x10
State.HEADER: 0x07
State.HEADER: 0x3d
State.HEADER: 0xbb
State.HEADER: 0x3a
State.HEADER: 0x49
State.HEADER: 0x9f
State.HEADER: 0x94
State.HEADER: 0xed
State.HEADER: 0xac
State.HEADER: 0x42
State.HEADER: 0x19
State.HEADER: 0x21
State.HEADER: 0x4d
State.HEADER: 0x2e
State.HEADER: 0x2e
State.HEADER: 0xe8
State.HEADER: 0xaf
State.HEADER: 0xc0
State.HEADER: 0xd1
State.HEADER: 0xf6
State.HEADER: 0x42
State.HEADER: 0xa3
State.HEADER: 0x8c
State.HEADER: 0x4f
State.HEADER: 0xdc
State.HEADER: 0x27
State.HEADER: 0xfd
State.HEADER: 0x0b
State.HEADER: 0x87
State.HEADER: 0xd1
State.HEADER: 0x04
State.HEADER: 0xfd
State.HEADER: 0x03
State.HEADER: 0x00
State.HEADER_CKSUM: 0x0b
State.HEADER_VALIDATE: 0x33
State.PAYLOAD: 0x01
State.PAYLOAD: 0x00
State.PAYLOAD: 0x01
State.CKSUM: 0x27
State.VALIDATE: 0x11


In [37]:
rng = np.random.default_rng()
src = uuid.uuid4()
dest = uuid.uuid4()

fileID = 0
seq = 1
ack = False

packet = E4E_Data_Raw_File_ACK(fileID, seq, ack, src, dest)
print(packet)
print(hex(len(packet.to_bytes())))
packets = parser.parseBytes(packet.to_bytes())
print(packets)
print(type(packets[0]))

E4EB 2B8F E3E1 5034 4722 81B1 3516 8749 2119 6F0A 19E2 CC14 4676 BABA 4CFD A158 0EEB 04FE 0500 7263 0100 0100 009D 61
0x2f
State.FIND_SYNC1: 0xe4
State.FIND_SYNC2: 0xeb
State.HEADER: 0x2b
State.HEADER: 0x8f
State.HEADER: 0xe3
State.HEADER: 0xe1
State.HEADER: 0x50
State.HEADER: 0x34
State.HEADER: 0x47
State.HEADER: 0x22
State.HEADER: 0x81
State.HEADER: 0xb1
State.HEADER: 0x35
State.HEADER: 0x16
State.HEADER: 0x87
State.HEADER: 0x49
State.HEADER: 0x21
State.HEADER: 0x19
State.HEADER: 0x6f
State.HEADER: 0x0a
State.HEADER: 0x19
State.HEADER: 0xe2
State.HEADER: 0xcc
State.HEADER: 0x14
State.HEADER: 0x46
State.HEADER: 0x76
State.HEADER: 0xba
State.HEADER: 0xba
State.HEADER: 0x4c
State.HEADER: 0xfd
State.HEADER: 0xa1
State.HEADER: 0x58
State.HEADER: 0x0e
State.HEADER: 0xeb
State.HEADER: 0x04
State.HEADER: 0xfe
State.HEADER: 0x05
State.HEADER: 0x00
State.HEADER_CKSUM: 0x72
State.HEADER_VALIDATE: 0x63
State.PAYLOAD: 0x01
State.PAYLOAD: 0x00
State.PAYLOAD: 0x01
State.PAYLOAD: 0x00
State.PAYLOAD:

In [38]:
rng = np.random.default_rng()
src = uuid.uuid4()
dest = uuid.uuid4()

fileID = 0
fileName = "Binary Data Protocol Specification.ipynb"
seq = 1
# Let us use a standard block size of 1024
with open(fileName, 'rb') as file:
    # Discard 0th block
    file.read(1024)
    # Keep 1st block
    blob = file.read(1024)
packet = E4E_Data_Raw_File(fileID, seq, blob, src, dest)
print(packet)
print(hex(len(packet.to_bytes())))
packets = parser.parseBytes(packet.to_bytes())
print(packets)
print(type(packets[0]))

E4EB D5A0 DABE A829 44F5 9269 0E4B 483E CAED E48E C106 452A 4DD5 84E9 45DA B2B8 E8A6 04FF 0C04 3BF2 0100 0100 0004 0000 0000 0000 2020 2020 7C22 0A20 2020 5D0A 2020 7D2C 0A20 207B 0A20 2020 2263 656C 6C5F 7479 7065 223A 2022 6D61 726B 646F 776E 222C 0A20 2020 226D 6574 6164 6174 6122 3A20 7B7D 2C0A 2020 2022 736F 7572 6365 223A 205B 0A20 2020 2022 2320 4865 6164 6572 2043 6865 636B 7375 6D5C 6E22 2C0A 2020 2020 2254 6865 2063 6865 636B 7375 6D20 7368 616C 6C20 6265 2063 616C 6375 6C61 7465 6420 6F76 6572 2074 6865 2065 6E74 6972 6574 7920 6F66 2074 6865 2070 6163 6B65 7420 6865 6164 6572 2028 6279 7465 7320 3078 3030 3030 2074 6F20 3078 3030 3235 2920 7573 696E 6720 7468 6520 4352 432D 3136 2063 6865 636B 7375 6D20 2870 6F6C 796E 6F6D 6961 6C20 3078 3130 3231 2C20 7374 6172 7469 6E67 2076 616C 7565 2030 7846 4646 462C 206E 6F20 7265 666C 6563 7469 6F6E 292E 2020 5468 6520 6368 6563 6B73 756D 2069 7473 656C 6620 7368 616C 6C20 6265 2073 746F 7265 6420 6173 2061 2031 3620 6269 7420 6269 

State.PAYLOAD: 0x20
State.PAYLOAD: 0x20
State.PAYLOAD: 0x20
State.PAYLOAD: 0x22
State.PAYLOAD: 0x6d
State.PAYLOAD: 0x65
State.PAYLOAD: 0x74
State.PAYLOAD: 0x61
State.PAYLOAD: 0x64
State.PAYLOAD: 0x61
State.PAYLOAD: 0x74
State.PAYLOAD: 0x61
State.PAYLOAD: 0x22
State.PAYLOAD: 0x3a
State.PAYLOAD: 0x20
State.PAYLOAD: 0x7b
State.PAYLOAD: 0x7d
State.PAYLOAD: 0x2c
State.PAYLOAD: 0x0a
State.PAYLOAD: 0x20
State.PAYLOAD: 0x20
State.PAYLOAD: 0x20
State.PAYLOAD: 0x22
State.PAYLOAD: 0x73
State.PAYLOAD: 0x6f
State.PAYLOAD: 0x75
State.PAYLOAD: 0x72
State.PAYLOAD: 0x63
State.PAYLOAD: 0x65
State.PAYLOAD: 0x22
State.PAYLOAD: 0x3a
State.PAYLOAD: 0x20
State.PAYLOAD: 0x5b
State.PAYLOAD: 0x0a
State.PAYLOAD: 0x20
State.PAYLOAD: 0x20
State.PAYLOAD: 0x20
State.PAYLOAD: 0x20
State.PAYLOAD: 0x22
State.PAYLOAD: 0x23
State.PAYLOAD: 0x20
State.PAYLOAD: 0x50
State.PAYLOAD: 0x61
State.PAYLOAD: 0x63
State.PAYLOAD: 0x6b
State.PAYLOAD: 0x65
State.PAYLOAD: 0x74
State.PAYLOAD: 0x20
State.PAYLOAD: 0x43
State.PAYLOAD: 0x68


In [39]:
import math
import threading
import filecmp

rng = np.random.default_rng()
txUUID = uuid.uuid4()
rxUUID = uuid.uuid4()
txQ = queue.Queue()
rxQ = queue.Queue()

outputFileName = "test.dat"

if os.path.isfile(outputFileName):
    os.remove(outputFileName)

fileName = "Binary Data Protocol Specification.ipynb"
mimeType = "application/x-ipynb+json"

class FileSenderState(enum.Enum):
    Header=1
    WaitForCTS=2
    Sending=3
    GetAcks=4
    
class FileReceiverState(enum.Enum):
    WaitForHeader=1
    GetData=2
    GetUnsent=3

def sender(fname, mimeType, blockSize=1024, fileID=0):
    state = FileSenderState.Header
    parser = binaryPacketParser()

    statResult = os.stat(fname)
    fileSize = statResult.st_size
    fileTime = dt.datetime.fromtimestamp(statResult.st_ctime)
    
    nPackets = math.ceil(fileSize / blockSize)
    sent = [False] * nPackets
    acked = [False] * nPackets
   
    with open(fname, 'rb') as fileData:
        while True:
            if state == FileSenderState.Header:

                packet = E4E_Data_Raw_File_Header(fileID, fname, mimeType, nPackets, fileTime, txUUID, rxUUID)
                rxQ.put(packet.to_bytes())
                state = FileSenderState.WaitForCTS
            elif state == FileSenderState.WaitForCTS:
                data = txQ.get()
                packets = parser.parseBytes(data)
                if len(packets):
                    if isinstance(packets[0], E4E_Data_Raw_File_CTS):
                        state = FileSenderState.Sending
                    else:
                        raise RuntimeError("Tx Didn't get CTS")
                else:
                    raise RuntimeError("Tx didn't get a packet")
            elif state == FileSenderState.Sending:
                fileData.seek(0)
                for seq in range(nPackets):
                    blob = fileData.read(blockSize)
                    if not acked[seq]:
                        packet = E4E_Data_Raw_File(fileID, seq, blob, txUUID, rxUUID)
                        rxQ.put(packet.to_bytes())
                    sent[seq] = True
                state = FileSenderState.GetAcks
            elif state == FileSenderState.GetAcks:
                data = txQ.get()
                packets = parser.parseBytes(data)
                if len(packets):
                    if isinstance(packets[0], E4E_Data_Raw_File_ACK):
                        packet = packets[0]
                        if packet.fileID == fileID:
                            acked[packet.seq] = packet.ack
                        if packet.ack == False:
                            state == FileSenderState.Sending
                if all(acked):
                    return
                        
def receiver(fname):
    state = FileReceiverState.WaitForHeader
    parser = binaryPacketParser()
    fileID = None
    nPackets = None
    received = []
    fileData = {}
    while True:
        if state == FileReceiverState.WaitForHeader:
            data = rxQ.get()
            packets = parser.parseBytes(data)
            if len(packets):
                if isinstance(packets[0], E4E_Data_Raw_File_Header):
                    packet = packets[0]
                    # generally here, we would write to the filename indicated by the packet, but for
                    # this example, I don't want to destroy existing files
                    fileID = packet.fileID
                    nPackets = packet.fileSize
                    fileTime = packet.fileTime
                    response = E4E_Data_Raw_File_CTS(fileID, True, rxUUID, txUUID)
                    txQ.put(response.to_bytes())
                    state = FileReceiverState.GetData
                    received = [False] * nPackets
        elif state == FileReceiverState.GetData:
            data = rxQ.get()
            packets = parser.parseBytes(data)
            for packet in packets:
                assert(isinstance(packet, E4E_Data_Raw_File))
                assert(packet.fileID == fileID)
                received[packet.seq] = True
                fileData[packet.seq] = packet.blob
                response = E4E_Data_Raw_File_ACK(fileID, packet.seq, True, rxUUID, txUUID)
                txQ.put(response.to_bytes())
            if all(received):
                with open(fname, 'wb') as file:
                    for i in range(nPackets):
                        file.write(fileData[i])
                return

transmitThread = threading.Thread(target=sender, args=(fileName, mimeType))
receiveThread = threading.Thread(target=receiver, args=(outputFileName,))

transmitThread.start()
receiveThread.start()

transmitThread.join()
receiveThread.join()

filecmp.cmp(fileName, outputFileName)

State.FIND_SYNC1: 0xe4
State.FIND_SYNC2: 0xeb
State.HEADER: 0x53
State.HEADER: 0xcd
State.HEADER: 0xae
State.HEADER: 0x81
State.HEADER: 0x47
State.HEADER: 0x5a
State.HEADER: 0x4d
State.HEADER: 0xe1
State.HEADER: 0xa9
State.HEADER: 0x43
State.HEADER: 0x3a
State.HEADER: 0xb0
State.HEADER: 0x27
State.HEADER: 0xf8
State.HEADER: 0x28
State.HEADER: 0x23
State.HEADER: 0xde
State.HEADER: 0x23
State.HEADER: 0xb7
State.HEADER: 0xdd
State.HEADER: 0x92
State.HEADER: 0x7f
State.HEADER: 0x43
State.HEADER: 0x31
State.HEADER: 0x92
State.HEADER: 0x17
State.HEADER: 0x8c
State.HEADER: 0xfa
State.HEADER: 0xc1
State.HEADER: 0xdc
State.HEADER: 0xf7
State.HEADER: 0xb2
State.HEADER: 0x04
State.HEADER: 0xfc
State.HEADER: 0x56
State.HEADER: 0x00
State.HEADER_CKSUM: 0xb0
State.HEADER_VALIDATE: 0x21
State.PAYLOAD: 0x01
State.PAYLOAD: 0x00
State.PAYLOAD: 0x28
State.PAYLOAD: 0x00
State.PAYLOAD: 0x18
State.PAYLOAD: 0x00
State.PAYLOAD: 0x11
State.PAYLOAD: 0x00
State.PAYLOAD: 0x00
State.PAYLOAD: 0x00
State.PAYLOAD: 0x

State.PAYLOAD: 0x68
State.PAYLOAD: 0x61
State.PAYLOAD: 0x72
State.PAYLOAD: 0x20
State.PAYLOAD: 0x28
State.PAYLOAD: 0x30
State.PAYLOAD: 0x78
State.PAYLOAD: 0x45
State.PAYLOAD: 0x34
State.PAYLOAD: 0x29
State.PAYLOAD: 0x20
State.PAYLOAD: 0x20
State.PAYLOAD: 0x20
State.PAYLOAD: 0x7c
State.PAYLOAD: 0x5c
State.PAYLOAD: 0x6e
State.PAYLOAD: 0x22
State.PAYLOAD: 0x2c
State.PAYLOAD: 0x0a
State.PAYLOAD: 0x20
State.PAYLOAD: 0x20
State.PAYLOAD: 0x20
State.PAYLOAD: 0x20
State.PAYLOAD: 0x22
State.PAYLOAD: 0x7c
State.PAYLOAD: 0x20
State.PAYLOAD: 0x30
State.PAYLOAD: 0x78
State.PAYLOAD: 0x30
State.PAYLOAD: 0x30
State.PAYLOAD: 0x30
State.PAYLOAD: 0x31
State.PAYLOAD: 0x20
State.PAYLOAD: 0x20
State.PAYLOAD: 0x20
State.PAYLOAD: 0x20
State.PAYLOAD: 0x20
State.PAYLOAD: 0x7c
State.PAYLOAD: 0x20
State.PAYLOAD: 0x55
State.PAYLOAD: 0x38
State.PAYLOAD: 0x20
State.PAYLOAD: 0x20
State.PAYLOAD: 0x20
State.PAYLOAD: 0x20
State.PAYLOAD: 0x20
State.PAYLOAD: 0x7c
State.PAYLOAD: 0x20
State.PAYLOAD: 0x53
State.PAYLOAD: 0x79


State.PAYLOAD: 0x20
State.PAYLOAD: 0x2b
State.PAYLOAD: 0x20
State.PAYLOAD: 0x4e
State.PAYLOAD: 0x20
State.PAYLOAD: 0x7c
State.PAYLOAD: 0x20
State.PAYLOAD: 0x55
State.PAYLOAD: 0x31
State.PAYLOAD: 0x36
State.PAYLOAD: 0x20
State.PAYLOAD: 0x20
State.PAYLOAD: 0x20
State.PAYLOAD: 0x20
State.PAYLOAD: 0x7c
State.PAYLOAD: 0x20
State.PAYLOAD: 0x50
State.PAYLOAD: 0x61
State.PAYLOAD: 0x63
State.PAYLOAD: 0x6b
State.PAYLOAD: 0x65
State.PAYLOAD: 0x74
State.PAYLOAD: 0x20
State.PAYLOAD: 0x43
State.PAYLOAD: 0x68
State.PAYLOAD: 0x65
State.PAYLOAD: 0x63
State.PAYLOAD: 0x6b
State.PAYLOAD: 0x73
State.PAYLOAD: 0x75
State.PAYLOAD: 0x6d
State.PAYLOAD: 0x20
State.PAYLOAD: 0x20
State.PAYLOAD: 0x20
State.PAYLOAD: 0x20
State.PAYLOAD: 0x7c
State.PAYLOAD: 0x5c
State.PAYLOAD: 0x6e
State.PAYLOAD: 0x22
State.PAYLOAD: 0x2c
State.PAYLOAD: 0x0a
State.PAYLOAD: 0x20
State.PAYLOAD: 0x20
State.PAYLOAD: 0x20
State.PAYLOAD: 0x20
State.PAYLOAD: 0x22
State.PAYLOAD: 0x7c
State.PAYLOAD: 0x20
State.PAYLOAD: 0x30
State.PAYLOAD: 0x78


State.PAYLOAD: 0x63
State.PAYLOAD: 0x6b
State.PAYLOAD: 0x73
State.PAYLOAD: 0x75
State.PAYLOAD: 0x6d
State.PAYLOAD: 0x20
State.PAYLOAD: 0x6f
State.PAYLOAD: 0x76
State.PAYLOAD: 0x65
State.PAYLOAD: 0x72
State.PAYLOAD: 0x20
State.PAYLOAD: 0x74
State.PAYLOAD: 0x68
State.PAYLOAD: 0x65
State.PAYLOAD: 0x20
State.PAYLOAD: 0x65
State.PAYLOAD: 0x6e
State.PAYLOAD: 0x74
State.PAYLOAD: 0x69
State.PAYLOAD: 0x72
State.PAYLOAD: 0x65
State.PAYLOAD: 0x74
State.PAYLOAD: 0x79
State.PAYLOAD: 0x20
State.PAYLOAD: 0x6f
State.PAYLOAD: 0x66
State.PAYLOAD: 0x20
State.PAYLOAD: 0x74
State.PAYLOAD: 0x68
State.PAYLOAD: 0x65
State.PAYLOAD: 0x20
State.PAYLOAD: 0x70
State.PAYLOAD: 0x61
State.PAYLOAD: 0x79
State.PAYLOAD: 0x6c
State.PAYLOAD: 0x6f
State.PAYLOAD: 0x61
State.PAYLOAD: 0x64
State.PAYLOAD: 0x2e
State.PAYLOAD: 0x22
State.PAYLOAD: 0x0a
State.PAYLOAD: 0x20
State.PAYLOAD: 0x20
State.PAYLOAD: 0x20
State.PAYLOAD: 0x5d
State.PAYLOAD: 0x0a
State.PAYLOAD: 0x20
State.PAYLOAD: 0x20
State.PAYLOAD: 0x7d
State.PAYLOAD: 0x2c


State.PAYLOAD: 0x7c
State.PAYLOAD: 0x20
State.PAYLOAD: 0x34
State.PAYLOAD: 0x20
State.PAYLOAD: 0x20
State.PAYLOAD: 0x20
State.PAYLOAD: 0x20
State.PAYLOAD: 0x20
State.PAYLOAD: 0x20
State.PAYLOAD: 0x20
State.PAYLOAD: 0x20
State.PAYLOAD: 0x20
State.PAYLOAD: 0x20
State.PAYLOAD: 0x20
State.PAYLOAD: 0x20
State.PAYLOAD: 0x20
State.PAYLOAD: 0x20
State.PAYLOAD: 0x7c
State.PAYLOAD: 0x20
State.PAYLOAD: 0x55
State.PAYLOAD: 0x6e
State.PAYLOAD: 0x73
State.PAYLOAD: 0x69
State.PAYLOAD: 0x67
State.PAYLOAD: 0x6e
State.PAYLOAD: 0x65
State.PAYLOAD: 0x64
State.PAYLOAD: 0x20
State.PAYLOAD: 0x33
State.PAYLOAD: 0x32
State.PAYLOAD: 0x20
State.PAYLOAD: 0x62
State.PAYLOAD: 0x69
State.PAYLOAD: 0x74
State.PAYLOAD: 0x20
State.PAYLOAD: 0x69
State.PAYLOAD: 0x6e
State.PAYLOAD: 0x74
State.PAYLOAD: 0x65
State.PAYLOAD: 0x67
State.PAYLOAD: 0x65
State.PAYLOAD: 0x72
State.PAYLOAD: 0x20
State.PAYLOAD: 0x20
State.PAYLOAD: 0x20
State.PAYLOAD: 0x20
State.PAYLOAD: 0x20
State.PAYLOAD: 0x20
State.PAYLOAD: 0x20
State.PAYLOAD: 0x20


...

State.PAYLOAD: 0x63
State.PAYLOAD: 0x65
State.PAYLOAD: 0x20
State.PAYLOAD: 0x73
State.PAYLOAD: 0x68
State.PAYLOAD: 0x61
State.PAYLOAD: 0x6c
State.PAYLOAD: 0x6c
State.PAYLOAD: 0x20
State.PAYLOAD: 0x65
State.PAYLOAD: 0x78
State.PAYLOAD: 0x70
State.PAYLOAD: 0x65
State.PAYLOAD: 0x63
State.PAYLOAD: 0x74
State.PAYLOAD: 0x20
State.PAYLOAD: 0x74
State.PAYLOAD: 0x68
State.PAYLOAD: 0x65
State.PAYLOAD: 0x20
State.PAYLOAD: 0x52
State.PAYLOAD: 0x61
State.PAYLOAD: 0x77
State.PAYLOAD: 0x20
State.PAYLOAD: 0x46
State.PAYLOAD: 0x69
State.PAYLOAD: 0x6c
State.PAYLOAD: 0x65
State.PAYLOAD: 0x20
State.PAYLOAD: 0x44
State.PAYLOAD: 0x61
State.PAYLOAD: 0x74
State.PAYLOAD: 0x61
State.PAYLOAD: 0x20
State.PAYLOAD: 0x70
State.PAYLOAD: 0x61
State.PAYLOAD: 0x63
State.PAYLOAD: 0x6b
State.PAYLOAD: 0x65
State.PAYLOAD: 0x74
State.PAYLOAD: 0x73
State.PAYLOAD: 0x20
State.PAYLOAD: 0x69
State.PAYLOAD: 0x6e
State.PAYLOAD: 0x20
State.PAYLOAD: 0x73
State.PAYLOAD: 0x65
State.PAYLOAD: 0x71
State.PAYLOAD: 0x75
State.PAYLOAD: 0x65


State.PAYLOAD: 0x36
State.PAYLOAD: 0x20
State.PAYLOAD: 0x20
State.PAYLOAD: 0x7c
State.PAYLOAD: 0x20
State.PAYLOAD: 0x50
State.PAYLOAD: 0x61
State.PAYLOAD: 0x63
State.PAYLOAD: 0x6b
State.PAYLOAD: 0x65
State.PAYLOAD: 0x74
State.PAYLOAD: 0x20
State.PAYLOAD: 0x53
State.PAYLOAD: 0x65
State.PAYLOAD: 0x71
State.PAYLOAD: 0x75
State.PAYLOAD: 0x65
State.PAYLOAD: 0x6e
State.PAYLOAD: 0x63
State.PAYLOAD: 0x65
State.PAYLOAD: 0x20
State.PAYLOAD: 0x20
State.PAYLOAD: 0x20
State.PAYLOAD: 0x20
State.PAYLOAD: 0x7c
State.PAYLOAD: 0x5c
State.PAYLOAD: 0x6e
State.PAYLOAD: 0x22
State.PAYLOAD: 0x2c
State.PAYLOAD: 0x0a
State.PAYLOAD: 0x20
State.PAYLOAD: 0x20
State.PAYLOAD: 0x20
State.PAYLOAD: 0x20
State.PAYLOAD: 0x22
State.PAYLOAD: 0x7c
State.PAYLOAD: 0x20
State.PAYLOAD: 0x30
State.PAYLOAD: 0x78
State.PAYLOAD: 0x30
State.PAYLOAD: 0x30
State.PAYLOAD: 0x30
State.PAYLOAD: 0x34
State.PAYLOAD: 0x20
State.PAYLOAD: 0x20
State.PAYLOAD: 0x20
State.PAYLOAD: 0x20
State.PAYLOAD: 0x20
State.PAYLOAD: 0x20
State.PAYLOAD: 0x20


State.VALIDATE: 0xd2
State.FIND_SYNC1: 0xe4
State.FIND_SYNC1: 0xe4State.FIND_SYNC2: 0xeb
State.HEADER: 0x53
State.HEADER: 0xcd
State.HEADER: 0xae
State.HEADER: 0x81
State.HEADER: 0x47
State.HEADER: 0x5a
State.HEADER: 0x4d
State.HEADER: 0xe1
State.HEADER: 0xa9
State.HEADER: 0x43
State.HEADER: 0x3a
State.HEADER: 0xb0
State.HEADER: 0x27
State.HEADER: 0xf8
State.FIND_SYNC2: 0xeb
State.HEADER: 0x28
State.HEADER: 0x23
State.HEADER: 0xde
State.HEADER: 0x23
State.HEADER: 0xb7
State.HEADER: 0xdd
State.HEADER: 0x92
State.HEADER: 0x7f
State.HEADER: 0x43
State.HEADER: 0x31

State.HEADER: 0x92
State.HEADER: 0x17
State.HEADER: 0x8c
State.HEADER: 0xfa
State.HEADER: 0xc1
State.HEADER: 0xdc
State.HEADER: 0xf7
State.HEADER: 0xb2
State.HEADER: 0x04
State.HEADER: 0xffState.HEADER: 0xde
State.HEADER: 0x23
State.HEADER: 0xb7
State.HEADER: 0xdd
State.HEADER: 0x92
State.HEADER: 0x7f
State.HEADER: 0x43
State.HEADER: 0x31
State.HEADER: 0x92
State.HEADER: 0x17

State.HEADER: 0x8c
State.HEADER: 0xfa
State.HEADER:

State.PAYLOAD: 0x3a
State.PAYLOAD: 0x20
State.PAYLOAD: 0x22
State.PAYLOAD: 0x69
State.PAYLOAD: 0x70
State.PAYLOAD: 0x79
State.PAYLOAD: 0x74
State.PAYLOAD: 0x68
State.PAYLOAD: 0x6f
State.PAYLOAD: 0x6e
State.PAYLOAD: 0x22
State.PAYLOAD: 0x2c
State.PAYLOAD: 0x0a
State.PAYLOAD: 0x20
State.PAYLOAD: 0x20
State.PAYLOAD: 0x20
State.PAYLOAD: 0x20
State.PAYLOAD: 0x22
State.PAYLOAD: 0x76
State.PAYLOAD: 0x65
State.PAYLOAD: 0x72
State.PAYLOAD: 0x73
State.PAYLOAD: 0x69
State.PAYLOAD: 0x6f
State.PAYLOAD: 0x6e
State.PAYLOAD: 0x22
State.PAYLOAD: 0x3a
State.PAYLOAD: 0x20
State.PAYLOAD: 0x33
State.PAYLOAD: 0x0a
State.PAYLOAD: 0x20
State.PAYLOAD: 0x20
State.PAYLOAD: 0x20
State.PAYLOAD: 0x7d
State.PAYLOAD: 0x2c
State.PAYLOAD: 0x0a
State.PAYLOAD: 0x20
State.PAYLOAD: 0x20
State.PAYLOAD: 0x20
State.PAYLOAD: 0x22
State.PAYLOAD: 0x66
State.PAYLOAD: 0x69
State.PAYLOAD: 0x6c
State.PAYLOAD: 0x65
State.PAYLOAD: 0x5f
State.PAYLOAD: 0x65
State.PAYLOAD: 0x78
State.PAYLOAD: 0x74
State.PAYLOAD: 0x65
State.PAYLOAD: 0x6e


True

In [42]:
data = bytes.fromhex("e4eb000000000000000000000000000000000000000000000000000000000000000004002e000100000000803f0000004000004040000080400000a0400000c0400000e04000000041000010410000")
parser = binaryPacketParser()
packets = parser.parseBytes(data)
packets


State.FIND_SYNC1: 0xe4
State.FIND_SYNC2: 0xeb
State.HEADER: 0x00
State.HEADER: 0x00
State.HEADER: 0x00
State.HEADER: 0x00
State.HEADER: 0x00
State.HEADER: 0x00
State.HEADER: 0x00
State.HEADER: 0x00
State.HEADER: 0x00
State.HEADER: 0x00
State.HEADER: 0x00
State.HEADER: 0x00
State.HEADER: 0x00
State.HEADER: 0x00
State.HEADER: 0x00
State.HEADER: 0x00
State.HEADER: 0x00
State.HEADER: 0x00
State.HEADER: 0x00
State.HEADER: 0x00
State.HEADER: 0x00
State.HEADER: 0x00
State.HEADER: 0x00
State.HEADER: 0x00
State.HEADER: 0x00
State.HEADER: 0x00
State.HEADER: 0x00
State.HEADER: 0x00
State.HEADER: 0x00
State.HEADER: 0x00
State.HEADER: 0x00
State.HEADER: 0x00
State.HEADER: 0x04
State.HEADER: 0x00
State.HEADER: 0x2e
State.HEADER: 0x00
State.HEADER_CKSUM: 0x01
State.HEADER_VALIDATE: 0x00
State.FIND_SYNC1: 0x00
State.FIND_SYNC1: 0x00
State.FIND_SYNC1: 0x00
State.FIND_SYNC1: 0x00
State.FIND_SYNC1: 0x00
State.FIND_SYNC1: 0x00
State.FIND_SYNC1: 0x00
State.FIND_SYNC1: 0x00
State.FIND_SYNC1: 0x00
State.FIND

[]

In [50]:
packetHeader = bytes.fromhex("00")
binascii.crc_hqx(packetHeader, 0xFFFF).to_bytes(2, "big").hex()

'e1f0'